## 用requests抓取链家网信息

![](./images/lianjia.png)

来动手做一个练习，做到学以致用。<br>
这次，我们来爬取链家网的一些内容，用的工具依旧是大家熟悉的requests和BeautifulSoup

### 1.准备工作

编写爬虫前的准备工作，我们需要导入用到的库，这里主要使用的是requests和lxml两个。还有一个Time库，负责设置每次抓取的休息时间。

In [1]:
import requests
import time

### 2.抓取列表页

开始抓取前当然应该了解一下目标网站URL结构咯。<br>
链家网的二手房列表页面共有100个，URL结构为http://bj.lianjia.com/ershoufang/pg9/ 
其中

- bj表示城市
- /ershoufang/是频道名称
- pg9是页面码。

我们要抓取的是北京的二手房频道，所以前面的部分不会变，属于固定部分，后面的页面码需要在1-100间变化，属于可变部分。将URL分为两部分，前面的固定部分赋值给url，后面的可变部分使用for循环遍历页面。

In [4]:
url = 'https://bj.lianjia.com/ershoufang/'
page = 'pg'

这里提一个小小的醒，我们最好在http请求中设置一个头部信息，否则很容易被封ip。头部信息网上有很多现成的，也可以使用httpwatch等工具来查看。

In [3]:
#设置请求头部信息
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
'Accept':'text/html;q=0.9,*/*;q=0.8',
'Accept-Charset':'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
'Accept-Encoding':'gzip',
'Connection':'close',
'Referer':'http://www.baidu.com/link?url=_andhfsjjjKRgEWkj7i9cFmYYGsisrnm2A-TN3XZDQXxvGsM9k9ZZSnikW2Yds4s&amp;amp;wd=&amp;amp;eqid=c3435a7d00146bd600000003582bfd1f'
}

我们使用for循环生成1-100的数字，转化格式后与前面的URL固定部分拼成要抓取的URL。这里我们设置每两个页面间隔0.5秒。抓取到的页面保存在html中。

In [5]:
for i in range(1,2):
    if i == 1:
        i = str(i)
        tmp_url = url+'/'
        r = requests.get(url=tmp_url, headers=headers)
        html = r.text
    else:
        i = str(i)
        tmp_url = url+page+i+'/'
        r = requests.get(url=tmp_url, headers=headers)
        html2 = r.text
        html += html2
    time.sleep(2)

### 页面解析
页面抓取的工作算是完成了，内容在html中，下一步就要进行页面解析了。我们依旧使用xml对页面进行解析。

In [6]:
from lxml import etree
selector = etree.HTML(html)

完成页面解析后就可以对页面中的关键信息进行提取了。下面我们分别对房源的总价，房源信息和关注度三部分进行提取。
把页面div标签中class=priceInfo的部分提取出来，并使用for循环将其中每个房源的总价数据存在tp中。

In [7]:
houses = selector.xpath('//li[@class="clear"]')

In [8]:
len(houses)

30

In [9]:
houses_title = []
houses_price = []
houses_info = []
followers_info = []

### 做一个统一的解析

In [14]:
for house in houses:
    house_title = house.xpath('.//div[@class="title"]/a/text()')[0]
    house_price = house.xpath('.//div[@class="totalPrice"]/span/text()')[0]
    house_info = house.xpath('.//div[@class="houseInfo"]/a/text()')[0]+house.xpath('.//div[@class="houseInfo"]/text()')[0]
    follower_info = house.xpath('.//div[@class="followInfo"]/text()')[0]
    
    houses_title.append(house_title)
    houses_price.append(house_price)
    houses_info.append(house_info)
    followers_info.append(follower_info)

In [15]:
house_dic = {'house_title':houses_title, 'house_price':houses_price, 'house_info':houses_info, 'follower_info':followers_info}

In [16]:
import pandas as pd
house_df = pd.DataFrame(house_dic)

In [17]:
house_df

,follower_info,house_info,house_price,house_title
0,82人关注 / 共40次带看 / 10天以前发布,翠微中里 | 2室1厅 | 64.04平米 | 南 北 | 简装 | 无电梯,605,翠微 财政部南北两居 中间层 一年不出一套 随时可看
1,104人关注 / 共31次带看 / 8天以前发布,西坝河北里 | 2室1厅 | 61.6平米 | 东南 西南 | 精装 | 有电梯,450,东南向两居室 高楼层采光好 看房方便
2,106人关注 / 共53次带看 / 15天以前发布,三里河路5号院 | 3室1厅 | 76.38平米 | 南 北 | 简装 | 有电梯,763,正规三居室 南北通透 带电梯 朝南无遮挡 看房方便
3,727人关注 / 共112次带看 / 14天以前发布,西直门北大街43号院 | 2室1厅 | 58.8平米 | 南 北 | 精装 | 无电梯,520,西直门通透户型 婚房装修 交通便捷 可改三居
4,170人关注 / 共41次带看 / 16天以前发布,西绒线 | 3室1厅 | 130.7平米 | 南 北 | 简装,1211,西单西绒线复式三居 单价低 户型方正 看房随时
5,108人关注 / 共40次带看 / 23天以前发布,西罗园四区 | 2室1厅 | 64.7平米 | 南 北 | 其他 | 无电梯,348,西罗园四区 南北通透两居 全明格局
6,192人关注 / 共62次带看 / 16天以前发布,天通西苑二区 | 2室1厅 | 85.57平米 | 南 | 精装 | 有电梯,380,西二区，商品房满5年唯一，厅南、双卧南、精装修
7,65人关注 / 共35次带看 / 16天以前发布,清缘东里 | 2室1厅 | 80.75平米 | 南 北 | 精装,580,清河，清缘东里，高层两居，视野好，诚售。
8,259人关注 / 共54次带看 / 19天以前发布,龙旗广场 | 2室1厅 | 72.13平米 | 西南 | 简装 | 有电梯,330,龙旗广场西南向两居室，看房方便，诚心出售
9,96人关注 / 共49次带看 / 18天以前发布,依翠园 | 4室2厅 | 124.13平米 | 东北 | 精装,580,依翠园新岚大厦四居室，诚心出售


来看看爬下来的**房价数据**
![](./images/house-price.jpg)

提取房源信息和关注度的方法与提取房源价格的方法类似，下面是具体的代码，房源信息存储在hi中，关注度存储在fi中。

来看看**房源信息**
![](./images/house-info.jpg)

再来看看关注度状况
![](./images/house-followed.jpg)

### 清洗数据并整理到数据表中

我们将之前爬取到的信息进行汇总，并导入pandas之中生成数据表。便于后面的分析。

很尴尬的是，大家看得到，很多信息是糊在一块的，不能直接使用，所以咱们再做一些数据提取和清洗的工作。如房源信息，在表中每个房源的小区名称，户型，面积，朝向等信息都在一个字段中，无法直接使用。需要先进行分列操作。这里的规则比较明显，每个信息间都是以竖线分割的，因此我们只需要以竖线进行分列即可。

现在再来看看我们整理好的数据

把拆分后的数据拼接回原始数据中

使用相同的方法对房源关注度字段进行分列和拼接操作。这里的分列规则是斜杠。

好了，这就是咱们对链家网的部分数据爬取过程了。<br>
跃跃欲试的宝宝们何不自己动手，再解析一些内容试试。<br>
另外对**数据分析和可视化**感兴趣的同学，千万不要错过咱们后面的案例哦，会有后续的跟进数据分析和展示~